In [3]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient


In [4]:
# Qdrant server URL
URL ="localhost"
# Qdrant dimension of the collection
DIMENSION = 384
# Qdrant collection name
COLLECTION_NAME = "data" #"incident data"
METRIC_NAME ="COSINE"

In [5]:
client = QdrantClient(URL,port=6333)

In [6]:
vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)

In [7]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

llm = OpenAI(api_base="http://localhost:1234/v1",
             api_key="lm-studio",
             #  model = 'lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf',
             )
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5", trust_remote_code=True)

Settings.llm = llm

In [8]:
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [9]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

In [10]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
    llm = llm,
)
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

In [12]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)
query = 'What measures are obliged entities required to implement for clients who are considered high risk in terms of money laundering and terrorism financing?'
# query
response = query_engine.query(query)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
print(str(response))

('According to the excerpt from "Opportunities and Policy Initiatives for '
 'Profitable Partnership and Infrastructure Investment in Indian Highways and '
 'National Highways Development Project: A Comprehensive Overview," '
 'contractors and supervision consultants from around 27 countries are already '
 "in India, helping implement one of the world's largest highways projects. "
 'The excerpt also provides a table that lists the number of contractors and '
 "consultants from each country that are currently working on India's highways "
 'projects. Therefore, foreign participation in highways in India is '
 'significant.')
